In [1]:
import torch
import torchvision
#from torchvision.datasets import MNIST
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline



In [2]:
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
trainset = torchvision.datasets.ImageFolder(root='/content/gdrive/My Drive/Int/mnist/training/', 
                                         transform=transforms.Compose([transforms.ToTensor(), 
                                                                       transforms.Grayscale(num_output_channels=1)]))

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root='/content/gdrive/My Drive/Int/mnist/testing/', 
                                         transform=transforms.Compose([transforms.ToTensor(), 
                                                                       transforms.Grayscale(num_output_channels=1)]))


testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=2)



image_tensor, label= testset[0]
print(image_tensor.shape, label)
#print(image_tensor[0][27][27])


torch.Size([1, 28, 28]) 0


In [4]:
class LinearReg(nn.Module):
  
  def __init__(self, in_size, out_size):
    super().__init__()
    self.lin = nn.Linear(in_features = in_size, out_features = out_size)
  
  def forward(self, X):
    pred = self.lin(X)
    return(pred)


In [5]:
torch.manual_seed(1)
model = LinearReg(784,1)
loss_fun = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)
[w,b] = model.parameters()

In [8]:
model.to(device)

LinearReg(
  (lin): Linear(in_features=784, out_features=1, bias=True)
)

In [10]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate (trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        X_train, y_train = data[0].to(device), data[1].to(device)
        X_train = X_train.reshape(4,784)
        #X_train = X_train.type(torch.scalar_tensor)

        y_train = y_train.reshape(4,1)
        y_train = y_train.type(torch.float)
        #print(X_train.shape, y_train.shape)



        # forward + backward + optimize
        y_pred = model.forward(X_train)
        loss = loss_fun(y_pred, y_train)
        #print("@epoch : ", i, " #Loss : ", loss.item())
        #losses.append(loss)


        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 0.585
[1,  4000] loss: 0.590
[1,  6000] loss: 0.603
[2,  2000] loss: 0.597
[2,  4000] loss: 0.631
[2,  6000] loss: 0.577
[3,  2000] loss: 0.607
[3,  4000] loss: 0.604
[3,  6000] loss: 0.586
[4,  2000] loss: 0.599
[4,  4000] loss: 0.630
[4,  6000] loss: 0.601
[5,  2000] loss: 0.572
[5,  4000] loss: 0.603
[5,  6000] loss: 0.603
[6,  2000] loss: 0.597
[6,  4000] loss: 0.596
[6,  6000] loss: 0.598
[7,  2000] loss: 0.577
[7,  4000] loss: 0.608
[7,  6000] loss: 0.608
[8,  2000] loss: 0.605
[8,  4000] loss: 0.601
[8,  6000] loss: 0.600
[9,  2000] loss: 0.603
[9,  4000] loss: 0.624
[9,  6000] loss: 0.588
[10,  2000] loss: 0.579
[10,  4000] loss: 0.624
[10,  6000] loss: 0.609
Finished Training


In [11]:
PATH = './MNISTLinearRegression.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
%ls

gdrive/  MNISTLinearRegression.pth  sample_data/


In [12]:
PATH = './MNISTLinearRegression.pth'
model = LinearReg(784,1)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [17]:
correct = 0
total = 0

with torch.no_grad():
    for data in  testloader:
        images, labels = data
        images = images.reshape(1,784)

        #labels = labels.reshape(1,1)
        #labels = labels.type(torch.float)
        

        outputs = model.forward(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        c = (predicted == labels).squeeze()
       
print('Accuracy of the model on the test data: %d %%' % (
    100 * correct / total))


Accuracy of the model on the test data: 19 %
